In [ ]:
import pandas as pd
import numpy as np

In [ ]:
date_csv = pd.read_csv("../input/kdd-cup-2015/ObjectData/date.csv")
date_csv.head(5)

In [ ]:
obj_csv = pd.read_csv("../input/kdd-cup-2015/ObjectData/object.csv")
obj_csv.head(5)

In [ ]:
train_path = "../input/kdd-cup-2015/train/"
test_path = "../input/kdd-cup-2015/test/"
enrollment_train_csv = pd.read_csv(train_path + "enrollment_train.csv")
log_train_csv = pd.read_csv(train_path + "log_train.csv")
truth_train_csv = pd.read_csv(train_path + "truth_train.csv")

enrollment_test_csv = pd.read_csv(test_path + "enrollment_train.csv")
log_test_csv = pd.read_csv(test_path + "log_train.csv")
truth_test_csv = pd.read_csv(test_path + "truth_train.csv")

In [ ]:
enrollment_train_csv.head()

In [ ]:
enrollment_test_csv.head()

In [ ]:
log_train_csv.head()

In [ ]:
truth_train_csv.head()

# **Problem Statement**

Given the data predict dropout rate of course. How many of the students leave course after enrolling into the course.

# **Data Preparation**

In [ ]:
dates_csv_path = "../input/kdd-cup-2015/ObjectData/date.csv"
enrollment_train_csv_path =train_path + "enrollment_train.csv"
log_train_csv_path = train_path + "log_train.csv"
truth_train_csv_path = train_path + "truth_train.csv"

enrollment_test_csv_path = test_path + "enrollment_train.csv"
log_test_csv_path = test_path + "log_train.csv"
truth_test_csv_path =test_path + "truth_train.csv"

In [ ]:
class DataCleaner:
    def __init__(self, dates, enrollment_train, log_train, enrollment_test, log_test, save_path):
        self.dates = dates_csv_path
        self.enrollment_train = enrollment_train_csv_path
        self.log_train =  pd.read_csv(log_train_csv_path)
        self.enrollment_test =  enrollment_test_csv_path
        self.log_test =  pd.read_csv(log_test_csv_path)
        self.save_path = "./"
    def Get_dict(self):
        dates_df = pd.read_csv(self.dates)
        course_map = pd.factorize(dates_df.course_id)[1]
        course_dict = dict(zip(course_map,range(len(course_map))))
        
        return course_dict 
    
    def Get_username_dict(self):
        source_path_train = self.enrollment_train
        source_path_test = self.enrollment_test 
        
        df = pd.read_csv(source_path_train,usecols=[1])
        username_map = pd.factorize(df.username)[1]
        username_dict = dict(zip(username_map,range(len(username_map))))
        
        df2 = pd.read_csv(source_path_test,usecols=[1])
        username_map2 = pd.factorize(df2.username)[1]
        diff = [w for w in username_map2 if w not in username_map]
        username_dict2 =dict(zip(diff,np.arange(len(username_map),len(username_map)+len(diff))))
        
        username_dict.update(username_dict2)
        print ("username_dict done...")
        return username_dict
    
    def course_map(self,x):
        return self.course_dict[x]
    
    def username_map(self,x):
        return self.username_dict[x]
    
    def clean_enrollments(self):
        train_df_enr = pd.read_csv(self.enrollment_train,usecols=[0,1,2],converters={1:self.username_map,2:self.course_map})
        train_df_enr.to_csv(self.save_path + "enrollment_train_final.csv",index=False)
   
        test_df_enr = pd.read_csv(self.enrollment_test,usecols=[0,1,2],converters={1:self.username_map,2:self.course_map})
        test_df_enr.to_csv(self.save_path + "enrollment_test_final.csv",index=False)
        
    def date_map(self):
        print ("read date.csv")
        source_path = self.dates
        df1 = pd.read_csv(source_path,converters={0:self.course_map})
        df1["day_nums"]= (pd.to_datetime(df1["to"]) - pd.to_datetime(df1["from"]))
        df1["day_nums"] = df1["day_nums"].map(lambda x: x.days)
        df1.to_csv(self.save_path + 'dates_final.csv',index=False)
                   
    def clean_logs(self):
        log_train_csv = self.log_train
        log_test_csv = self.log_test
        
        log_train_csv["date"] = log_train_csv["time"].map(lambda d: d[:10])
        log_train_csv["time"] = log_train_csv["time"].map(lambda t: t[11:])
        log_train_csv.to_csv(self.save_path + "log_train_final.csv",index=False)
        
        log_test_csv["date"] = log_test_csv["time"].map(lambda d: d[:10])
        log_test_csv["time"] = log_test_csv["time"].map(lambda t: t[11:])
        log_test_csv.to_csv(self.save_path + "log_test_final.csv",index=False)   
        
    def merge_enr_date(self, train_data_final, test_data_final, date_final):
        train_df_enr = pd.read_csv(train_data_final)
        test_df_enr = pd.read_csv(test_data_final)
        date_df = pd.read_csv(date_final)
        
        merge_df_tr_ = pd.merge(train_df_enr, date_df, how="left",left_on="course_id",right_on="course_id")
        
        merge_df_ts = pd.merge(test_df_enr, date_df, how="left",on="course_id")
        
        merge_df_tr_.to_csv(self.save_path + "merged_enrollment_date_train.csv", index = False)
        merge_df_ts.to_csv(self.save_path + "merged_enrollment_date_test.csv", index = False)
    
    def merge_enr_log(self,train_data_final,test_data_final, train_log_final, test_log_final):
        train_df_enr = pd.read_csv(train_data_final)
        train_log_final = pd.read_csv(train_log_final)
        merged_df_tr = pd.merge(train_df_enr,train_log_final,how="left",on="enrollment_id")
        merged_df_tr["interval"]= (pd.to_datetime(merged_df_tr["date"]) - pd.to_datetime(merged_df_tr["from"]))
        merged_df_tr["interval"] = merged_df_tr["interval"].map(lambda x: x.days+1)
        merged_df_tr.drop(["from"],axis=1,inplace=True)
        merged_df_tr.to_csv(self.save_path + "merged_enrollment_log_traints.csv",index=False)
        
        test_df_enr = pd.read_csv(test_data_final)
        test_log_final = pd.read_csv(test_log_final)
        merged_df_ts = pd.merge(test_df_enr,train_log_final,how="left",on="enrollment_id")
        merged_df_ts["interval"]= (pd.to_datetime(merged_df_ts["date"]) - pd.to_datetime(merged_df_ts["from"]))
        merged_df_ts["interval"] = merged_df_ts["interval"].map(lambda x: x.days+1)
        merged_df_ts.drop(["from"],axis=1,inplace=True)
        merged_df_ts.to_csv(self.save_path + "merged_enrollment_log_test.csv",index=False)
        
    def data_trans(self):
        self.course_dict = self.Get_dict()
        self.username_dict = self.Get_username_dict()
        self.clean_enrollments()
        self.date_map()
        self.clean_logs()
        
        train_data_final = "./enrollment_train_final.csv"
        test_data_final = "./enrollment_test_final.csv"
        date_final = "./dates_final.csv"
        self.merge_enr_date(train_data_final, test_data_final, date_final)
        
        train_log_final = "./log_train_final.csv"
        test_log_final = "./log_test_final.csv"
        train_data_final_d = "./merged_enrollment_date_train.csv"
        test_data_final_d = "./merged_enrollment_date_test.csv"
        self.merge_enr_log(train_data_final_d, test_data_final_d, train_log_final, test_log_final)

In [ ]:
p = DataCleaner(dates_csv_path, enrollment_train_csv_path, log_train_csv_path, enrollment_test_csv_path, log_test_csv_path, save_path="./")  
p.data_trans()

In [ ]:
pd.read_csv("./dates_final.csv")

In [ ]:
train = pd.read_csv("./merged_enrollment_date_train.csv")
truth = pd.read_csv("../input/kdd-cup-2015/train/truth_train.csv",names=['enrollment_id','dropout'])

In [ ]:
train.head()

In [ ]:
truth.head()

In [ ]:
train_truth_merged = pd.merge(train, truth, how="left",on="enrollment_id")

In [ ]:
train_truth_merged.head()

In [ ]:
gbby_username_df = train_truth_merged.groupby("username").course_id.count().to_frame()
gbby_username_df.rename(columns={'course_id':'course_num'}, inplace = True)
gbby_username_df.head()

In [ ]:
gbby_username_dropout_df = train_truth_merged.groupby(["username","dropout"]).course_id.count().unstack().fillna(0)
gbby_username_dropout_df.rename(columns={0:'nondropout_num', 1:'dropout_num'}, inplace = True)
gbby_username_dropout_df.drop(["dropout_num"],axis=1,inplace=True)
gbby_username_dropout_df

In [ ]:
train_gbby_username_df_merged = pd.merge(train,gbby_username_df,how="left",left_on="username",right_index=True)
train_gbby_username_df_merged.head()

In [ ]:
train_gbby_username_df_merged = pd.merge(train_gbby_username_df_merged,gbby_username_dropout_df,how="left",left_on="username",right_index=True)
train_gbby_username_df_merged.head()

In [ ]:
target_path_train = "./truth_train_merged_final.csv"
truth_train_gbby_username_df_merged = pd.merge(train_gbby_username_df_merged,truth,how="left",on="enrollment_id")
truth_train_gbby_username_df_merged.to_csv(target_path_train,index=False)

In [ ]:
truth_train_gbby_username_df_merged.head()

In [ ]:
test = pd.read_csv("./merged_enrollment_date_test.csv")
test.head()

In [ ]:
test_gbby_username_df_merged = pd.merge(test,gbby_username_df,how="left",left_on="username",right_index=True)
test_gbby_username_df_merged.head()

In [ ]:
target_path_test = "./truth_test_merged_final.csv"

test_final = pd.merge(test_gbby_username_df_merged,gbby_username_dropout_df,how="left",left_on="username",right_index=True)
test_final.fillna(0,inplace=True)
test_final.to_csv(target_path_test,index=False)

In [ ]:
test_final.head()

# **EDA**

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib as plt 
import seaborn as sns 

In [ ]:
train_data = pd.read_csv("./truth_train_merged_final.csv")

In [ ]:
train_data.head()

In [ ]:
train_data.dtypes

In [ ]:
train_data.shape

In [ ]:
train_data.describe()

In [ ]:
train_data.value_counts().to_frame()

In [ ]:
train_data.nunique().to_frame()

In [ ]:
sns.countplot(train_data.dropout)

In [ ]:
train_data[(train_data['dropout'] == 0)].describe()

In [ ]:
train_data[(train_data['dropout'] == 1)].describe()

In [ ]:
train_data.apply(lambda x: sum(x.isnull()),axis=0)
# we already removed it earlier

In [ ]:
#left skewed
train_data['nondropout_num'].hist(bins=25)

In [ ]:
# left skew
train_data['course_num'].hist(bins=20)

In [ ]:
train_data['day_nums'].hist(bins=30)

In [ ]:
print(train_data.boxplot(column='course_num'))

In [ ]:
print(train_data.boxplot(column='day_nums'))

In [ ]:
print(train_data.boxplot(column='username',by = 'dropout'))

In [ ]:
df1 = pd.read_csv("./merged_enrollment_log_traints.csv")

In [ ]:
df1

In [ ]:
gpby_course = df1.groupby("course_id").username.count()

In [ ]:
gpby_course.head().to_frame()

In [ ]:
gpby_enrol = df1.groupby("enrollment_id")
# grouping the data according to enrollment id i.e sare enr_id = 1 ek ke niche ek

In [ ]:
gpby_enrol.head(5)

In [ ]:
enrol_list = list()
for enrollment_id,group in gpby_enrol:
    i = enrollment_id
    x = group
    c = group.groupby("interval")
    for interval,group2 in group.groupby('interval'):
        y = interval
        z = group2
        enrol_list.append(enrollment_id)

In [ ]:
i

In [ ]:
x

In [ ]:
c.head()

In [ ]:
y

In [ ]:
z

In [ ]:
pd.DataFrame(enrol_list)

In [ ]:
def add(x,y):
        return x+y    
        
def op_character(source_path, target_path):
    print("op_character...")
    df1 = pd.read_csv(source_path)
    gpby_enrol = df1.groupby("enrollment_id")

    enrol_list = list()
    interval_list = list()
    last_minutes = list()
    valid_opnum = list()
    all_opnum = list()

    for enrollment_id,group in gpby_enrol:
        group.groupby("interval")
        for interval,group2 in group.groupby('interval'):
            enrol_list.append(enrollment_id)

            interval_list.append(interval)
            timelist = group2.time.tolist()
            h1 = datetime.datetime.strptime(timelist[0],'%H:%M:%S')
            h2 = datetime.datetime.strptime(timelist[len(timelist)-1],'%H:%M:%S')
            hh = h2-h1
            last_minutes.append(hh.seconds/60+1)
            valid_len = [0,0,0,0]
            valid_len[0] = len(group2[group2.event=='problem'])
            valid_len[1] = len(group2[group2.event=='video'])
            valid_len[2] = len(group2[group2.event == 'wiki'])
            valid_len[3] = len(group2[group2.event == 'discussion'])
            valid_opnum.append(reduce(add,valid_len))
            all_opnum.append(len(group2))
    df2 = pd.DataFrame({"enrollment_id":enrol_list,"interval":interval_list,"last_minutes":last_minutes,"valid_opnum":valid_opnum,"all_opnum":all_opnum})
    df2 = df2[["enrollment_id","interval","last_minutes","valid_opnum","all_opnum"]] 
    df2.to_csv(target_path,index=False)

In [ ]:
op_character("./merged_enrollment_log_traints.csv", "./")